# Analyse/Validate Radon "Raw" Results (LOC, etc.)

In [1]:
import pandas as pd
import numpy as np
import sys
from os.path import join
import altair as alt # Python wrapper for Vega-Lite visualisation grammarimport altair as alt

In [2]:
DATA_DIR = "../output/"

In [3]:
raw_df_py2 = pd.read_csv(join(DATA_DIR, "results_radon_raw_python2.csv"))

In [4]:
raw_df_py3 = pd.read_csv(join(DATA_DIR, "results_radon_raw_python3.csv"))

In [5]:
raw_df_py2

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
0,72225778,72225778/predict.py,138.0,100.0,113.0,2.0,0.0,23.0,2.0,False,False
1,72225778,72225778/text_cnn_rnn.py,98.0,68.0,69.0,9.0,0.0,21.0,8.0,False,False
2,72225778,72225778/data_helper.py,112.0,100.0,94.0,3.0,0.0,17.0,1.0,False,False
3,72225778,72225778/train.py,160.0,100.0,123.0,12.0,0.0,25.0,12.0,False,False
4,118248283,118248283/train_speech_commands.py,278.0,193.0,211.0,12.0,0.0,58.0,9.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...
7936,88318858,88318858/batchgen.py,136.0,90.0,86.0,13.0,3.0,34.0,13.0,False,False
7937,88318858,88318858/train.py,247.0,159.0,166.0,51.0,0.0,43.0,38.0,False,False
7938,40576835,40576835/get_accuracies.py,79.0,55.0,55.0,3.0,0.0,21.0,3.0,False,False
7939,40576835,40576835/api_benchmarks.py,245.0,163.0,187.0,8.0,0.0,50.0,8.0,False,False


In [6]:
raw_df_py3

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
0,72225778,72225778/predict.py,138.0,100.0,113.0,2.0,0.0,23.0,2.0,False,False
1,72225778,72225778/text_cnn_rnn.py,98.0,68.0,69.0,9.0,0.0,21.0,8.0,False,False
2,72225778,72225778/data_helper.py,112.0,100.0,94.0,3.0,0.0,17.0,1.0,False,False
3,72225778,72225778/train.py,160.0,100.0,123.0,12.0,0.0,25.0,12.0,False,False
4,118248283,118248283/train_speech_commands.py,278.0,193.0,211.0,12.0,0.0,58.0,9.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...
7936,88318858,88318858/batchgen.py,136.0,90.0,86.0,13.0,3.0,34.0,13.0,False,False
7937,88318858,88318858/train.py,247.0,159.0,166.0,51.0,0.0,43.0,38.0,False,False
7938,40576835,40576835/get_accuracies.py,79.0,55.0,55.0,3.0,0.0,21.0,3.0,False,False
7939,40576835,40576835/api_benchmarks.py,245.0,163.0,187.0,8.0,0.0,50.0,8.0,False,False


# Self-consistency of LOC calculations

According to https://radon.readthedocs.io/en/latest/commandline.html#the-raw-command "The equation sloc+multi+singlecomments+blank=loc should always hold."

In [7]:
target_loc3 = raw_df_py3.sloc + raw_df_py3.multi + raw_df_py3.single_comments + raw_df_py3.blank
target_loc2 = raw_df_py2.sloc + raw_df_py2.multi + raw_df_py2.single_comments + raw_df_py2.blank

In [8]:
# Should return no results if equation holds
raw_df_py2[(raw_df_py2["loc"] != target_loc2) & ~pd.isnull(raw_df_py2["loc"])]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error


In [9]:
# Should return no results if equation holds
raw_df_py3[(raw_df_py3["loc"] != target_loc3) & ~pd.isnull(raw_df_py3["loc"])]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error


# Discrepancies in reported LOC between different Python versions

In [10]:
all_loc_diffs = (raw_df_py2["loc"] != raw_df_py3["loc"]) & ~(pd.isnull(raw_df_py2["loc"]) & pd.isnull(raw_df_py3["loc"]))
all_loc_diffs_nonull = (raw_df_py2["loc"] != raw_df_py3["loc"]) & ~(pd.isnull(raw_df_py2["loc"]) | pd.isnull(raw_df_py3["loc"]))

In [11]:
raw_df_py2[all_loc_diffs_nonull]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
15,118248283,118248283/models/resnext.py,141.0,80.0,79.0,0.0,39.0,23.0,0.0,False,False
22,93305304,93305304/data.py,58.0,39.0,33.0,7.0,4.0,14.0,7.0,False,False
23,93305304,93305304/eval.py,54.0,36.0,31.0,8.0,4.0,12.0,7.0,False,False
24,93305304,93305304/modules.py,289.0,112.0,143.0,12.0,109.0,32.0,5.0,False,False
25,93305304,93305304/hyperparams.py,34.0,21.0,19.0,15.0,4.0,5.0,6.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...
7870,1383879,1383879/simplenlp/ja_cabocha/word.py,459.0,350.0,260.0,3.0,33.0,103.0,63.0,False,False
7871,1383879,1383879/simplenlp/ja_cabocha/properties.py,48.0,31.0,29.0,0.0,6.0,12.0,1.0,False,False
7883,32617150,32617150/setup.py,24.0,2.0,23.0,0.0,0.0,1.0,0.0,False,False
7884,32617150,32617150/docs/conf.py,357.0,31.0,54.0,218.0,0.0,93.0,210.0,False,False


In [12]:
raw_df_py3[all_loc_diffs_nonull]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
15,118248283,118248283/models/resnext.py,142.0,80.0,79.0,1.0,39.0,23.0,1.0,False,False
22,93305304,93305304/data.py,59.0,39.0,33.0,8.0,4.0,14.0,8.0,False,False
23,93305304,93305304/eval.py,55.0,36.0,31.0,9.0,4.0,12.0,8.0,False,False
24,93305304,93305304/modules.py,290.0,112.0,143.0,13.0,109.0,32.0,6.0,False,False
25,93305304,93305304/hyperparams.py,35.0,21.0,19.0,16.0,4.0,5.0,7.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...
7870,1383879,1383879/simplenlp/ja_cabocha/word.py,460.0,350.0,260.0,4.0,33.0,103.0,64.0,False,False
7871,1383879,1383879/simplenlp/ja_cabocha/properties.py,49.0,31.0,29.0,1.0,6.0,12.0,2.0,False,False
7883,32617150,32617150/setup.py,25.0,2.0,23.0,1.0,0.0,1.0,1.0,False,False
7884,32617150,32617150/docs/conf.py,359.0,31.0,54.0,220.0,0.0,93.0,212.0,False,False


There are differences between the `loc` calculation in Radon for Python2 versus Radon for Python3.
The differences seem to occur in files that include `# -*- coding: utf-8 -*-`. In contrast, the `sloc` (Source Lines Of Code) metric seems to be more stable.

# Discrepancies in reported SLOC

In [13]:
all_sloc_diffs = (raw_df_py2["sloc"] != raw_df_py3["sloc"]) & ~(pd.isnull(raw_df_py2["sloc"]) & pd.isnull(raw_df_py3["sloc"]))
all_sloc_diffs_nonull = (raw_df_py2["sloc"] != raw_df_py3["sloc"]) & ~(pd.isnull(raw_df_py2["sloc"]) | pd.isnull(raw_df_py3["sloc"]))

In [14]:
raw_df_py2[all_sloc_diffs]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
4372,590142,590142/everything_else/djfrontend/django-1.0.2...,676.0,451.0,410.0,49.0,92.0,116.0,58.0,False,False
4422,35187829,35187829/processors/persons.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4424,35187829,35187829/processors/attribs.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4425,35187829,35187829/processors/entities.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4437,35187829,35187829/queries/builtin.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4439,35187829,35187829/queries/arithmetic.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4443,35187829,35187829/queries/bus.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4444,35187829,35187829/queries/wiki.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4454,35187829,35187829/utils/cmp_parse.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4459,35187829,35187829/utils/cmp.py,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False


In [15]:
raw_df_py3[all_sloc_diffs]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
4372,590142,590142/everything_else/djfrontend/django-1.0.2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4422,35187829,35187829/processors/persons.py,501.0,216.0,285.0,53.0,54.0,94.0,68.0,False,False
4424,35187829,35187829/processors/attribs.py,159.0,58.0,60.0,25.0,27.0,41.0,31.0,False,False
4425,35187829,35187829/processors/entities.py,571.0,287.0,331.0,81.0,24.0,121.0,95.0,False,False
4437,35187829,35187829/queries/builtin.py,1133.0,469.0,732.0,147.0,37.0,197.0,167.0,False,False
4439,35187829,35187829/queries/arithmetic.py,498.0,171.0,315.0,39.0,20.0,118.0,45.0,False,False
4443,35187829,35187829/queries/bus.py,814.0,300.0,531.0,104.0,31.0,134.0,118.0,False,False
4444,35187829,35187829/queries/wiki.py,178.0,68.0,92.0,15.0,17.0,49.0,20.0,False,False
4454,35187829,35187829/utils/cmp_parse.py,157.0,121.0,119.0,31.0,0.0,18.0,20.0,False,False
4459,35187829,35187829/utils/cmp.py,1440.0,964.0,1171.0,332.0,32.0,99.0,138.0,False,False


Radon appears to be able to calculate raw results on some Python2 files even when run as Python3 (as it just needs to extract the tokens, not build a full AST). However, there were still some cases where the correct version of Python was needed for Radon to extract raw stats for the file.

In [16]:
raw_df_py2[all_sloc_diffs_nonull]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
5624,3287642,3287642/redisbayes.py,231.0,79.0,136.0,24.0,39.0,32.0,24.0,False,False


raw_df_py3[all_sloc_diffs_nonull]

There seems to be one minor difference in whether a comment is treated as a multi-line string between Python2 and Python3 versions of Radon, but otherwise the results are consistent.

## Misc

Confirm that large files processed correctly:

In [21]:
raw_df_py3[raw_df_py3["path"] == "3546355/preprocess/punkt_data_german.py"]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
1771,3546355,3546355/preprocess/punkt_data_german.py,60318.0,7.0,60312.0,1.0,0.0,5.0,1.0,False,False


In [22]:
raw_df_py2[raw_df_py2["path"] == "3546355/preprocess/punkt_data_german.py"]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
1771,3546355,3546355/preprocess/punkt_data_german.py,60317.0,7.0,60312.0,0.0,0.0,5.0,0.0,False,False


In [24]:
raw_df_py3[raw_df_py3["path"] == "15985050/src/feature/ner_PER_list.py"]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
3687,15985050,15985050/src/feature/ner_PER_list.py,57071.0,1.0,57070.0,1.0,0.0,0.0,1.0,False,False


In [25]:
raw_df_py2[raw_df_py2["path"] == "15985050/src/feature/ner_PER_list.py"]

,repo,path,loc,lloc,sloc,comments,multi,blank,single_comments,parse_error,internal_error
3687,15985050,15985050/src/feature/ner_PER_list.py,57070.0,1.0,57070.0,0.0,0.0,0.0,0.0,False,False


(seems fine)